## Imports

In [60]:
import torch
from torch import nn
from torch import Tensor
import torch.optim as optim
from torchvision import models, transforms, datasets
from tqdm import tqdm

torch.cuda.is_available()

True

In [61]:
import matplotlib.pyplot as plt
import numpy as np
import glob
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import cv2

In [62]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


## Visualize the data

In [63]:
def plot_accuracy_from_history(history):
    plt.plot(history['accuracy'], label='accuracy')
    plt.plot(history['val_accuracy'], label = 'val_accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.ylim([0, 1])
    plt.legend(loc='lower right')
    plt.show()

# Transfer Learning

## Prepare Data

In [64]:
EPOCHS: int = 10

In [65]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(150),
        transforms.CenterCrop(150),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'validation': transforms.Compose([
        transforms.Resize(150),
        transforms.CenterCrop(150),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(150),
        transforms.CenterCrop(150),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
}

# Define data directories
data_dir: str = 'kaggle/cats_and_dogs_small/'
image_datasets = {x: datasets.ImageFolder(root=data_dir+x, transform=data_transforms[x]) for x in ['train', 'validation', 'test']}
dataloaders = {x: DataLoader(image_datasets[x], batch_size=20, shuffle=False) for x in ['train', 'validation', 'test']}


## Features Extraction Method (faster but no augmentation)

### Define ResNet Model

In [66]:
class ResNetModel(nn.Module):
    def __init__(self):
        super(ResNetModel, self).__init__()
        self.resnet = models.resnet18(pretrained=True)
        self.resnet.fc = nn.Identity()  # Remove the final fully connected layer

    def forward(self, x):
        x = self.resnet(x)
        return x

### Extract features from ResNet

In [67]:
def extract_features(model, dataloader, dataset_size) -> tuple[torch.Tensor, torch.Tensor]:
    model.eval()
    features = torch.zeros(dataset_size, 512, 4, 4, device=device)
    labels = torch.zeros(dataset_size, device=device)
    with torch.no_grad():
        for i, (inputs, labels_batch) in enumerate(dataloader):
            inputs = inputs.to(device)  # Move inputs to the same device as the model
            outputs = model(inputs)
            # Reshape or unsqueeze outputs to match the shape of features
            outputs = torch.unsqueeze(torch.unsqueeze(outputs, -1), -1)
            features[i * dataloader.batch_size : (i + 1) * dataloader.batch_size] = outputs
            labels[i * dataloader.batch_size : (i + 1) * dataloader.batch_size] = labels_batch
    return features, labels

In [68]:
resnet_model = ResNetModel().to(device)

train_features, train_labels = extract_features(resnet_model, dataloaders['train'], len(image_datasets['train']))
validation_features, validation_labels = extract_features(resnet_model, dataloaders['validation'], len(image_datasets['validation']))
test_features, test_labels = extract_features(resnet_model, dataloaders['test'], len(image_datasets['test']))

# Reshape
train_features = train_features.view(len(image_datasets['train']), -1)
validation_features = validation_features.view(len(image_datasets['validation']), -1)
test_features = test_features.view(len(image_datasets['test']), -1)

print(train_features.shape)
print(validation_features.shape)
print(test_features.shape)

/home/lenovo/Proj/DeepNeuralNetworks/.venv/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/lenovo/Proj/DeepNeuralNetworks/.venv/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


torch.Size([8000, 8192])
torch.Size([1000, 8192])
torch.Size([2000, 8192])


In [69]:
train_dataset = torch.utils.data.TensorDataset(train_features, train_labels)
validation_dataset = torch.utils.data.TensorDataset(validation_features, validation_labels)
test_dataset = torch.utils.data.TensorDataset(test_features, test_labels)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=20, shuffle=True)
validation_loader = torch.utils.data.DataLoader(validation_dataset, batch_size=20, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=20, shuffle=True)

### New model

In [70]:
class ModelWithExtracted(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(train_features.shape[1], 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 1),
            nn.Sigmoid(),
            nn.Flatten(0, 1)
        )

    def forward(self, x):
        x = self.model(x)
        return x

### Train

In [75]:
model1 = ModelWithExtracted().to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model1.parameters(), lr=0.001)

def train_model(model, criterion, optimizer) -> None:
    for epoch in range(EPOCHS):
        model.train()
        running_loss = 0.0
        running_corrects = 0
        for inputs, labels in tqdm(train_loader):
            inputs = inputs.to(device)
            labels = labels.float().to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum((outputs > 0.5) == labels.byte())
        epoch_loss = running_loss / len(image_datasets['train'])
        epoch_acc = running_corrects / len(image_datasets['train'])
        
        inputs_val, labels_val = next(iter(validation_loader))
        with torch.no_grad():
            model.eval()
            inputs_val = inputs_val.to(device)
            labels_val = labels_val.float().to(device)
            val_outputs = model(inputs_val)
            val_loss = criterion(val_outputs, labels_val)
            val_preds = torch.round(val_outputs)
            val_acc = (val_preds == labels_val).float().mean()
        
            print(f"Epoch {epoch+1}/{EPOCHS} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")
            print(f"Validation Loss: {val_loss:.4f} Acc: {val_acc:.4f}")
        
train_model(model1, criterion, optimizer)

  0%|          | 0/400 [00:00<?, ?it/s]

100%|██████████| 400/400 [00:05<00:00, 71.82it/s]


Epoch 1/10 Loss: 0.2811 Acc: 0.9495
Validation Loss: 0.0044 Acc: 1.0000


100%|██████████| 400/400 [00:04<00:00, 84.49it/s]


Epoch 2/10 Loss: 0.0894 Acc: 0.9678
Validation Loss: 0.0362 Acc: 1.0000


...

Epoch 8/10 Loss: 0.0721 Acc: 0.9700
Validation Loss: 0.0108 Acc: 1.0000


100%|██████████| 400/400 [00:04<00:00, 93.61it/s]


Epoch 9/10 Loss: 0.0658 Acc: 0.9741
Validation Loss: 0.0003 Acc: 1.0000


100%|██████████| 400/400 [00:04<00:00, 90.08it/s]

Epoch 10/10 Loss: 0.0680 Acc: 0.9756
Validation Loss: 0.0376 Acc: 1.0000


## Fine-Tuning method (slower but with augmentation)

### Prepare Data

In [94]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(150),
        transforms.CenterCrop(150),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        # # random horizontal flip
        # transforms.RandomHorizontalFlip(),
        # # random Linear Transformation
        # transforms.RandomAffine(degrees=20, translate=(0.1, 0.1)),
    ]),
    'validation': transforms.Compose([
        transforms.Resize(150),
        transforms.CenterCrop(150),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(150),
        transforms.CenterCrop(150),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
}

# Define data directories
data_dir = 'kaggle/cats_and_dogs_small/'
image_datasets = {x: datasets.ImageFolder(root=data_dir+x, transform=data_transforms[x]) for x in ['train', 'validation', 'test']}
dataloaders = {x: DataLoader(image_datasets[x], batch_size=20, shuffle=False) for x in ['train', 'validation', 'test']}

### Create model

In [95]:
class ModelFineTuning(nn.Module):
    def __init__(self):
        super().__init__()
        self.resnet = models.resnet18(pretrained=True)
        for param in self.resnet.parameters(): # freeze the ResNet layers
            param.requires_grad = False
        self.resnet.fc = nn.Identity()
        self.model = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 1),
            nn.Sigmoid(),
            nn.Flatten(0, 1)
        )

    def forward(self, x):
        x = self.resnet(x)
        x = self.model(x)
        return x
    
    
model2 = ModelFineTuning().to(device)
# Define loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.RMSprop(model2.parameters(), lr=2e-5)

In [96]:
# Training loop
for epoch in range(EPOCHS):
    model2.train()
    running_loss = 0.0
    running_corrects = 0
    for inputs, labels in tqdm(dataloaders['train']):
        optimizer.zero_grad()
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        outputs = model2(inputs)
        loss = criterion(outputs, labels.float())
        loss.backward()
        optimizer.step()
        running_loss += loss.cpu().item() * inputs.size(0)
        running_corrects += torch.sum((outputs > 0.5) == labels.byte())
    epoch_loss = running_loss / len(image_datasets['train'])
    epoch_acc = running_corrects / len(image_datasets['train'])

    # Validation loop
    model2.eval()
    val_running_loss = 0.0
    val_running_corrects = 0
    with torch.no_grad():
        for inputs, labels in tqdm(dataloaders['validation']):
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            outputs = model2(inputs)
            val_loss = criterion(outputs, labels.float())
            val_running_loss += val_loss.cpu().item() * inputs.size(0)
            val_running_corrects += torch.sum((outputs > 0.5) == labels.byte())
    val_epoch_loss = val_running_loss / len(image_datasets['validation'])
    val_epoch_acc = val_running_corrects / len(image_datasets['validation'])

    print(f'Epoch {epoch+1}/{EPOCHS} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
    print(f'Validation Loss: {val_epoch_loss:.4f} Acc: {val_epoch_acc:.4f}')

100%|██████████| 50/50 [00:06<00:00,  7.34it/s]


Epoch 1/10 Loss: 0.3151 Acc: 0.8939
Validation Loss: 1.7855 Acc: 0.5000


 62%|██████▏   | 248/400 [00:44<00:27,  5.58it/s]


KeyboardInterrupt: 

# Tuning Method